In [1]:
import tensorflow as tf

In [2]:
a = tf.constant(1)

In [3]:
a.graph is tf.get_default_graph()

True

+ tf.get_variable() & tf.Variable()
+ tf.Graph()
+ tf.variable_scope()

In [4]:
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("Var", shape=(1,2,3), initializer=tf.zeros_initializer())

In [5]:
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable("Var", shape=(2,3,2), initializer=tf.ones_initializer())

In [6]:
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("Var")))    

[[[0. 0. 0.]
  [0. 0. 0.]]]


In [7]:
with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("Var")))

[[[1. 1.]
  [1. 1.]
  [1. 1.]]

 [[1. 1.]
  [1. 1.]
  [1. 1.]]]


In [8]:
g = tf.Graph()
with g.as_default():
    zero = tf.Variable(initial_value=tf.zeros(shape=[1]), name='Var')
    one = tf.Variable(initial_value=tf.ones(shape=[1,2,3]), name='Var1')
    
with tf.Session(graph=g) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(zero))
    print(sess.run(one))

[0.]
[[[1. 1. 1.]
  [1. 1. 1.]]]


In [9]:
a = tf.constant([[1,2,3],[1,2,3],[1,2,3],[1,2,3]], name='a')
b = tf.constant([[3,4,5],[3,4,5],[3,4,5],[3,4,5]], name='b')

result = tf.add(a,b, name='add')
print(result)

Tensor("add:0", shape=(4, 3), dtype=int32)


In [10]:
result.get_shape()

TensorShape([Dimension(4), Dimension(3)])

+ tf.Tensor.eval() / sess.run(t)

In [11]:
t = tf.constant([1,2,3], name="test")

In [12]:
sess = tf.Session()
with sess.as_default():
    print(t.eval())
    print(sess.run(t))

[1 2 3]
[1 2 3]


In [13]:
sess = tf.Session()
print(t.eval(session=sess))

[1 2 3]


In [14]:
test = tf.constant([1,2,3], name='test')

In [15]:
sess = tf.Session()
with sess.as_default():
    print(sess is tf.get_default_session())
    
    print(test.eval() == sess.run(test))
    print(test.eval())
    print(sess.run(test))

True
[ True  True  True]
[1 2 3]
[1 2 3]


**`sess.run() 可以一步获取多个tensor中的值`**

In [16]:
a = tf.constant([1, 2, 3], name='a')
b = tf.constant([3, 4, 5], name='b')

In [17]:
add = tf.add(a, b, name='add')
mul = tf.multiply(a, b, name='mul')

In [18]:
with tf.Session() as sess:
    print(sess.run([add, mul]))
    print(add.eval(session=sess), mul.eval(session=sess))

[array([4, 6, 8]), array([ 3,  8, 15])]
[4 6 8] [ 3  8 15]


In [19]:
ones = tf.ones([2,3,4])
rand = tf.random_normal([2,3,4], mean=10, stddev=10)

var = tf.Variable(initial_value=ones)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(var))

[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]


In [23]:
w1 = tf.Variable(tf.random_normal([3,2], stddev=1))
w2 = tf.Variable(tf.random_normal([1,3], stddev=1))

x = tf.placeholder(tf.float32, shape=(2,1), name="input")
a = tf.matmul(w1, x)
y = tf.matmul(w2, a)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run(y, feed_dict={
        x : [[0.7],[0.9]]
    })
    print(res)

[[-0.93353766]]


In [24]:
w1 = tf.Variable(tf.random_normal([3,2], stddev=1))
w2 = tf.Variable(tf.random_normal([1,3], stddev=1))

x = tf.placeholder(tf.float32, shape=(2,3), name="input")
a = tf.matmul(w1, x)
y = tf.matmul(w2, a)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run(y, feed_dict={
        x : [[0.7,0.5,0.5],[0.9,0.4,0.8]]
    })
    print(res)

[[-1.6142296 -0.8062438 -1.3774065]]


In [27]:
w1 = tf.Variable(tf.random_normal([3,2], stddev=1))
w2 = tf.Variable(tf.random_normal([1,3], stddev=1))

x = tf.placeholder(tf.float32, shape=(2,3), name="input")
a = tf.matmul(w1, x)
y = tf.matmul(w2, a)

y_ = [[-1.5, -0.5, 1]]
cross_entropy = tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
learning_rate = 0.001
train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(cross_entropy)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run(y, feed_dict={
        x : [[0.7,0.5,0.5],[0.9,0.4,0.8]]
    })
    print(res)
    print(tf.trainable_variables())

[[0.25851858 0.11242405 0.23139453]]
[<tf.Variable 'Variable:0' shape=(2, 3, 4) dtype=float32_ref>, <tf.Variable 'Variable_1:0' shape=(3, 2) dtype=float32_ref>, <tf.Variable 'Variable_2:0' shape=(1, 3) dtype=float32_ref>, <tf.Variable 'Variable_3:0' shape=(3, 2) dtype=float32_ref>, <tf.Variable 'Variable_4:0' shape=(1, 3) dtype=float32_ref>, <tf.Variable 'Variable_5:0' shape=(3, 2) dtype=float32_ref>, <tf.Variable 'Variable_6:0' shape=(1, 3) dtype=float32_ref>, <tf.Variable 'Variable_7:0' shape=(3, 2) dtype=float32_ref>, <tf.Variable 'Variable_8:0' shape=(1, 3) dtype=float32_ref>, <tf.Variable 'Variable_9:0' shape=(3, 2) dtype=float32_ref>, <tf.Variable 'Variable_10:0' shape=(1, 3) dtype=float32_ref>, <tf.Variable 'Variable_11:0' shape=(3, 2) dtype=float32_ref>, <tf.Variable 'Variable_12:0' shape=(1, 3) dtype=float32_ref>, <tf.Variable 'Variable_13:0' shape=(3, 2) dtype=float32_ref>, <tf.Variable 'Variable_14:0' shape=(1, 3) dtype=float32_ref>]


In [60]:
import numpy as np
from numpy.random import RandomState

batch_size=8

w1 = tf.Variable(tf.random_normal([3,2],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([1,3],stddev=1,seed=1))

x = tf.placeholder(tf.float32,shape=(2,None),name='x_input')
y_ = tf.placeholder(tf.float32,shape=(1,None),name ='y_input')

a = tf.matmul(w1,x)
y = tf.matmul(w2,a)

#tf.clip_by_value()可以将计算的数值限制在一个范围内（1e-10~1.0）
#y_表示真实值，y表示预测值，定义的是交叉熵损失函数
#对于回归问题，最常用的损失函数是均方误差（MSE）mse = tf.reduce_mean(tf.square(y_-y))
cross_entropy = -tf.reduce_mean(
            y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))
#多分类问题适合softmax+cross_entrpy
#cross_entropy2 = tf.nn.softmax_cross_entropy_with_logits(y,y_)
train_step = tf.train.AdamOptimizer(0.015).minimize(cross_entropy)

rdm = RandomState(1)
dataset_size=128
X = rdm.rand(2, dataset_size)
Y = np.array([int(x1+x2<1) for (x1,x2) in X.T]).reshape(1,-1)
print(X[:,0])

with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    print (sess.run(w1))
    print (sess.run(w2))
    
    steps = 5000
    for i in range(steps):
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        sess.run(train_step,feed_dict = {x:X[:,start:end],y_:Y[:,start:end]})
        if i%1000==0:
            total_cross_entropy = sess.run(
                cross_entropy,feed_dict={x:X,y_:Y})
            print("After %d training_step(s) ,cross_entropy on all data is %g"%(i,total_cross_entropy))
    print (sess.run(w1))
    print (sess.run(w2))

[0.417022   0.12417332]
[[-0.8113182   1.4845988 ]
 [ 0.06532937 -2.4427042 ]
 [ 0.0992484   0.5912243 ]]
[[-0.8113182   1.4845988   0.06532937]]
After 0 training_step(s) ,cross_entropy on all data is 10.1598
After 1000 training_step(s) ,cross_entropy on all data is 0.0161879
After 2000 training_step(s) ,cross_entropy on all data is 0.00604948
After 3000 training_step(s) ,cross_entropy on all data is 0.00330034
After 4000 training_step(s) ,cross_entropy on all data is 0.000244847
[[-3.165786   -0.65562004]
 [ 0.8128642  -1.2751489 ]
 [ 5.766369    3.5891197 ]]
[[-3.9340394   0.38874954  3.9193282 ]]


In [53]:
import tensorflow as tf
from numpy.random import RandomState

batch_size=8

w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x = tf.placeholder(tf.float32,shape=(None,2),name='x_input')
y_ = tf.placeholder(tf.float32,shape=(None,1),name ='y_input')

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

#tf.clip_by_value()可以将计算的数值限制在一个范围内（1e-10~1.0）
#y_表示真实值，y表示预测值，定义的是交叉熵损失函数
#对于回归问题，最常用的损失函数是均方误差（MSE）mse = tf.reduce_mean(tf.square(y_-y))
cross_entropy = -tf.reduce_mean(
            y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))
#多分类问题适合softmax+cross_entrpy
#cross_entropy2 = tf.nn.softmax_cross_entropy_with_logits(y,y_)
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

rdm = RandomState(1)
dataset_size=128
X = rdm.rand(dataset_size,2)
print(X)
Y = [[int(x1+x2<1)] for (x1,x2) in X]
print(Y)
with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    print (sess.run(w1))
    print (sess.run(w2))
    
    steps = 5000
    for i in range(steps):
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        
        sess.run(train_step,feed_dict = {x:X[start:end],y_:Y[start:end]})
        if i%1000==0:
            total_cross_entropy = sess.run(
                cross_entropy,feed_dict={x:X,y_:Y})
            print("After %d training_step(s) ,cross_entropy on all data is %g"%(i,total_cross_entropy))
    print (sess.run(w1))
    print (sess.run(w2))

[[4.17022005e-01 7.20324493e-01]
 [1.14374817e-04 3.02332573e-01]
 [1.46755891e-01 9.23385948e-02]
 [1.86260211e-01 3.45560727e-01]
 [3.96767474e-01 5.38816734e-01]
 [4.19194514e-01 6.85219500e-01]
 [2.04452250e-01 8.78117436e-01]
 [2.73875932e-02 6.70467510e-01]
 [4.17304802e-01 5.58689828e-01]
 [1.40386939e-01 1.98101489e-01]
 [8.00744569e-01 9.68261576e-01]
 [3.13424178e-01 6.92322616e-01]
 [8.76389152e-01 8.94606664e-01]
 [8.50442114e-02 3.90547832e-02]
 [1.69830420e-01 8.78142503e-01]
 [9.83468338e-02 4.21107625e-01]
 [9.57889530e-01 5.33165285e-01]
 [6.91877114e-01 3.15515631e-01]
 [6.86500928e-01 8.34625672e-01]
 [1.82882773e-02 7.50144315e-01]
 [9.88861089e-01 7.48165654e-01]
 [2.80443992e-01 7.89279328e-01]
 [1.03226007e-01 4.47893526e-01]
 [9.08595503e-01 2.93614148e-01]
 [2.87775339e-01 1.30028572e-01]
 [1.93669579e-02 6.78835533e-01]
 [2.11628116e-01 2.65546659e-01]
 [4.91573159e-01 5.33625451e-02]
 [5.74117605e-01 1.46728575e-01]
 [5.89305537e-01 6.99758360e-01]
 [1.023344